In [ ]:
# ! pip install beautifulsoup4
# ! pip install requests

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
TOP200DATA = '/content/drive/MyDrive/AWSHACK/spotifytopsongsdata.csv'
TOP200WITHLYRICSDATA = '/content/drive/MyDrive/AWSHACK/spotifytopsongswithlyricsdata.csv'
TOP200WITHAZLYRICSDATA = '/content/drive/MyDrive/AWSHACK/spotifytopsongswithlyricsdata-az.csv'

In [ ]:
df = pd.read_csv(TOP200DATA)

In [ ]:
df_artist_track = df[['Track Name', 'Artist']].drop_duplicates()

In [ ]:
def common_formater(word):
  word = str(word)
  word = word.replace(".", "")
  word = word.replace(",", "")
  word = word.replace('\\', "")
  word = word.replace("/", " ")
  word = word.replace("Ã©", "e")
  word = word.replace("Ã‰", "e")
  word = word.replace("Ãº", "u")
  word = word.replace("$$", "")
  word = word.replace("$", " ")
  word = word.replace("Ã", "a")
  word = word.replace("Ã¡", "a")
  word = word.replace("Ã£", "a")
  word = word.replace("'", "")
  word = word.replace("!", "")
  word = word.replace(":", "")
  word = word.replace("â€™", "")
  word = word.replace("Ã˜", "")
  word = word.replace('&', 'and')
  return word.lower()

In [ ]:
def clean_songname(songname):
  songname = common_formater(songname)
  songname = re.split('[-()]+', songname)[0].rstrip().split("feat")[0].rstrip()
  return str(songname.replace(' ','-')) if ' ' in songname else str(songname)

In [ ]:
clean_songname('Ayala (Outro)')

'ayala'

In [ ]:
def clean_artistname(artistname):
  artistname = common_formater(artistname)
  artistname = artistname[0].upper() + artistname[1:]
  return str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)

In [ ]:
clean_artistname('Steve Aoki')

'Steve-aoki'

In [ ]:
def scrape_geniuslyrics(row):
    artistname = str(row['Artist'])
    songname = str(row['Track Name'])
    lyrics = str(row['lyrics'])
    lyrics_text = ''
    if lyrics == '' or lyrics == None:
      artistname = clean_artistname(artistname)
      songname = clean_songname(songname)
      print("Downloading Lyrics for: ",songname, artistname)
      page = requests.get('https://genius.com/'+ artistname + '-' + songname + '-' + 'lyrics')
      html = BeautifulSoup(page.text, 'html.parser')
      lyrics = html.find_all("div", class_="Lyrics__Container-sc-1ynbvzw-6")
      if len(lyrics) > 0:
        print("Downloaded Lyrics for: ",songname)
      for l in lyrics:
        x= l.get_text("\n")
        lyrics_text += "".join(re.split("\(|\)|\[|\]", x)[::2])
      lyrics_text = " ".join(lyrics_text.split('\n'))
    else:
      lyrics_text = lyrics
    return lyrics_text

In [ ]:
df_artist_track['lyrics'] = df_artist_track.apply(scrape_geniuslyrics, axis=1)

Downloaded Lyrics for:  please-shut-up
Downloaded Lyrics for:  plain-jane
Downloaded Lyrics for:  plain-jane-remix
Downloaded Lyrics for:  cocky
Downloaded Lyrics for:  
Downloaded Lyrics for:  bad-company
Downloaded Lyrics for:  a-ap-forever
Downloaded Lyrics for:  praise-the-lord
Downloaded Lyrics for:  a-ap-forever-remix
Downloaded Lyrics for:  distorted-records
Downloaded Lyrics for:  fukk-sleep
Downloaded Lyrics for:  tony-tone
Downloaded Lyrics for:  calldrops
Downloaded Lyrics for:  buck-shots
Downloaded Lyrics for:  gunz-n-butter
Downloaded Lyrics for:  og-beeper
Downloaded Lyrics for:  brotha-man
Downloaded Lyrics for:  purity
Downloaded Lyrics for:  kids-turned-out-fine
Downloaded Lyrics for:  hun43rd
Downloaded Lyrics for:  changes
Downloaded Lyrics for:  black-tux-white-collar
Downloaded Lyrics for:  sundress
Downloaded Lyrics for:  jet-lag
Downloaded Lyrics for:  babushka-boi
Downloaded Lyrics for:  move-ya-hips
Downloaded Lyrics for:  rich-nigga-problems


In [ ]:
def scrape_azlyrics(row):
  artistname = common_formater(row['Artist']).replace(' ','')
  songname = common_formater(row['Track Name']).replace(' ','')
  songname = re.split('[-()]+', songname)[0].strip().split("feat")[0].strip()
  page = requests.get('https://www.azlyrics.com/lyrics/'+ artistname + '/' + songname + '.html')
  try:
    html = BeautifulSoup(page.text, 'html.parser')
    divs = html.find('div', class_='col-xs-12 col-lg-8 text-center')
    lyric_div = divs.find('div', class_=None)
    itags = lyric_div.find_all('i')
    for itag in itags:
      itag.decompose()
    text = lyric_div.text
    text = re.sub('[()?]', '', text)
    text = " ".join(list(set(text.split('\n'))))
    print('lyrics saved for:', songname)
    return text.strip()
  except AttributeError:
    print('could not find lyrics for:', artistname, songname)
    return ''

In [ ]:
df_artist_track['lyrics'] = df_artist_track.apply(scrape_azlyrics, axis=1)

could not find lyrics for: theweeknd starboy
could not find lyrics for: thechainsmokers closer
could not find lyrics for: djsnake letmeloveyou
could not find lyrics for: cleanbandit rockabye
could not find lyrics for: drake onedance
could not find lyrics for: drake fakelove
could not find lyrics for: brunomars 24kmagic
could not find lyrics for: maroon5 dontwannaknow
could not find lyrics for: raesremmurd blackbeatles
could not find lyrics for: theweeknd ifeelitcoming
could not find lyrics for: shakira chantaje
could not find lyrics for: zayn idontwannaliveforever
could not find lyrics for: jamesarthur sayyouwontletgo
could not find lyrics for: migos badandboujee
could not find lyrics for: majorlazer coldwater
could not find lyrics for: martingarrix inthenameoflove
could not find lyrics for: arianagrande sidetoside
could not find lyrics for: starley callonme
could not find lyrics for: justintimberlake cantstopthefeeling
could not find lyrics for: thechainsmokers dontletmedown
could not

KeyboardInterrupt: ignored

In [ ]:
scrape_azlyrics({
    "Artist": "linkinpark",
    "Track Name": "numb"
})

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="noindex,nofollow" name="robots"/>
<title>AZLyrics - request for access</title>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" rel="stylesheet"/>
<link href="https://code.jquery.com/ui/1.12.1/themes/base/jquery-ui.min.css" rel="stylesheet"/>
<link href="/bsaz.css" rel="stylesheet"/>
<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script>
      <script src="https://oss.maxcdn.com/respond/1.4.2/respond.min.js"></script>
    <![endif]-->
<script async="" defer="" src="https://www.google.com/recaptcha/api.js"></scri

''

In [ ]:
df_artist_track.to_csv(TOP200WITHAZLYRICSDATA)